In [4]:
%iam_role arn:aws:iam::458226961977:role/glue_exec_role
%region ap-northeast-2
%connections steadio-segment-con

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is None
iam_role has been set to arn:aws:iam::458226961977:role/glue_exec_role.
Previous region: None
Setting new region to: ap-northeast-2
Region is set to: ap-northeast-2
Connections to be included:
steadio-segment-con


In [0]:
import sys
from awsglue import DynamicFrame
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext
from pyspark.sql.functions import udf

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::458226961977:role/glue_exec_role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 1981d9ee-1376-4b40-b0e9-7bc121def690
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
Waiting for session 1981d9ee-1376-4b40-b0e9-7bc121def690 to get into ready status...
Session 1981d9ee-1376-4b40-b0e9-7bc121def690 has been created




In [1]:
glue_context = GlueContext(SparkContext.getOrCreate())
spark = glue_context.spark_session
job = Job(glue_context)

username = "admin_user"
database = "dev"
password = "Hg6nNbUe24ud6WwM"
url = "jdbc:redshift://steadio-segment.cdftxbqtzhu1.ap-northeast-2.redshift.amazonaws.com:5439/dev"

In [22]:
source_dlab_ods_streak_steadio_outreach =glue_context.create_dynamic_frame.from_options(
    connection_type = "redshift",
    connection_options = {
                            "url": url
                            ,"database" : database
                            ,"user" : username
                            ,"password" : password
                            ,"query":"""
                                    SELECT assignedtosharingentries AS email
                                        , fields
                                        , stagekey AS stage_type_id
                                        , createddate AS created_at
                                    FROM tumblbug_dw.streak.boxes
                                    WHERE pipelineid = 'agxzfm1haWxmb29nYWVyMwsSDE9yZ2FuaXphdGlvbiIMdHVtYmxidWcuY29tDAsSCFdvcmtmbG93GICAxoW9psQLDA'
                                    """
                            ,"redshiftTmpDir" : "s3://steadio-glue-info/temporary/"
                            ,"connectionName":"steadio-segment-con-jdbc"
                            ,"aws_iam_role": "arn:aws:iam::458226961977:role/redshift-exec-role"
                            }
                        )

In [23]:
source_dlab_ods_streak_steadio_outreach.toDF().show()
source_dlab_ods_streak_steadio_outreach.toDF().count()

+--------------------+--------------------+-------------+--------------------+
|               email|              fields|stage_type_id|          created_at|
+--------------------+--------------------+-------------+--------------------+
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5001|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-05-11 10:01:...|
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5005|2022-06-08 01:40:...|
|[{"displayName":"...|{"1006":{"calcula...|         5001|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         5003|2022-06-08 02:04:...|
|[{"displayName":"...|{"1006":{"calcula...|         

In [92]:
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, BooleanType


def parse_json(json_val, key:str, separator:str=None, idx:int=None):
    if not json_val:
        return None
    
    dic_val = json.loads(json_val)
    if (isinstance(dic_val, list)):
        dic_val = dic_val[0]

    if not separator:
        try:
            if isinstance(dic_val[key], str):
                return int(dic_val[key])
            return dic_val[key]
        except ValueError:
            return dic_val[key]          
        except KeyError:
            return None
    try:
        return dic_val[key].split(separator)[idx]
    except:
        raise

udf_email = udf(lambda x: parse_json(json_val=x, key="email", separator='@', idx=0),StringType())
udf_cat = udf(lambda x: parse_json(json_val=x, key="1003"),IntegerType())
udf_tumblbug = udf(lambda x: parse_json(json_val=x, key="1014"),BooleanType())
udf_lead = udf(lambda x: parse_json(json_val=x, key="1002"),IntegerType())

In [93]:
df_dlab_ods_streak_steadio_outreach = source_dlab_ods_streak_steadio_outreach.toDF()
parsed_dlab_ods_streak_steadio_outreach = df_dlab_ods_streak_steadio_outreach.\
                                            withColumn("rep_name", udf_email(df_dlab_ods_streak_steadio_outreach["email"])).\
                                            withColumn("category_id", udf_cat(df_dlab_ods_streak_steadio_outreach["fields"])).\
                                            withColumn("is_tumblbug", udf_tumblbug(df_dlab_ods_streak_steadio_outreach["fields"])).\
                                            withColumn("leadsource_id", udf_lead(df_dlab_ods_streak_steadio_outreach["fields"]))

### Debug

In [77]:
import json
from pyspark.sql.functions import udf, size
from pyspark.sql.types import StringType, IntegerType, BooleanType, ArrayType

def parse_json(json_val, key:str, separator:str=None, idx:int=None):
    if not json_val:
        return None
    
    dic_val = json.loads(json_val)
    if (isinstance(dic_val, list)) & (len(dic_val) == 1):
        dic_val = dic_val[0]
       
    if not separator:
        try:
            return dic_val[key]
        except KeyError:
            return None
    try:
        return dic_val[key].split(separator)[idx]
    except:
        return '에러 발생'

udf_email = udf(lambda x: parse_json(json_val=x, key="email", separator='@', idx=0),StringType())
udf_cat = udf(lambda x: parse_json(json_val=x, key="1003"),StringType())
udf_tumblbug = udf(lambda x: parse_json(json_val=x, key="1014"),StringType())
udf_lead = udf(lambda x: parse_json(json_val=x, key="1002"),StringType())

In [40]:
parsed_dlab_ods_streak_steadio_outreach.createOrReplaceTempView("DATA")
spark.sql("SELECT fields, is_tumblbug FROM DATA where category_id IS NOT NULL").show()

+--------------------+-----------+
|              fields|is_tumblbug|
+--------------------+-----------+
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       true|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       true|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       true|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
|{"1006":{"calcula...|       null|
+--------------------+-----------+
only showing top 20 rows


In [34]:
temp_df = spark.sql("SELECT fields, category_id FROM DATA where category_id IS NOT NULL")
row_list = temp_df.collect()
print(row_list[0].__getitem__(0))
print("-"*50)
temp_df = spark.sql("SELECT fields, category_id FROM DATA where category_id IS NULL")
row_list = temp_df.collect()
print(row_list[0].__getitem__(0))

{"1006":{"calculationValue":0.0,"calculationValueType":"java.lang.Double","creationTimestamp":1650959268187,"lastUpdatedTimestamp":1650959268187,"lastCalculatedTimestamp":1651111962381,"calculationStatus":"SUCCESS"},"1014":true}
--------------------------------------------------
{"1006":{"calculationValue":0.0,"calculationValueType":"java.lang.Double","creationTimestamp":1654653845559,"lastUpdatedTimestamp":1654653845559,"lastCalculatedTimestamp":1655797741782,"calculationStatus":"SUCCESS"},"1003":"9004"}


In [76]:
parsed_dlab_ods_streak_steadio_outreach.createOrReplaceTempView("DATA")
temp_df = spark.sql("SELECT email FROM DATA where rep_name = '에러 발생'")
row_list = temp_df.collect()
print(row_list[0])

Row(email='[{"displayName":"(Sean) 김","fullName":"김철민(Sean)","email":"sean@tumblbug.com","image":"https://lh3.googleusercontent.com/a-/AOh14Gg9oPjs_ebuTz-rDbZFvmoNQNfbTiMKOmjJz1A4nw=s96-c","userKey":"agxzfm1haWxmb29nYWVyLwsSDE9yZ2FuaXphdGlvbiIMdHVtYmxidWcuY29tDAsSBFVzZXIYgIDc4J2u_woM"},{"displayName":"(june) 권","fullName":"권지연(june)","email":"june@tumblbug.com","image":"https://lh3.googleusercontent.com/a-/AOh14GjsLq-PPsnuJHm2ybfusY_-I_w_Tki7x_z39ydZ=s96-c","userKey":"agxzfm1haWxmb29nYWVyLwsSDE9yZ2FuaXphdGlvbiIMdHVtYmxidWcuY29tDAsSBFVzZXIYgICqgty23woM"}]')


----

In [94]:
print("Total row counts:")
print(pre_dlab_ods_streak_steadio_outreach.count())
print("Total counts of unique tuples:")
pre_dlab_ods_streak_steadio_outreach.distinct().count()

Total row counts:
1136
Total counts of unique tuples:
1095


In [95]:
pre_dlab_ods_streak_steadio_outreach = parsed_dlab_ods_streak_steadio_outreach.\
                                        select("rep_name", "category_id", "stage_type_id", "is_tumblbug", "leadsource_id", "created_at")

pre_dlab_ods_streak_steadio_outreach.createOrReplaceTempView("DATA")
spark.sql("SELECT DISTINCT rep_name FROM DATA").show()
spark.sql("SELECT DISTINCT category_id FROM DATA").show()
spark.sql("SELECT DISTINCT stage_type_id FROM DATA").show()
spark.sql("SELECT DISTINCT is_tumblbug FROM DATA").show()
spark.sql("SELECT DISTINCT leadsource_id FROM DATA").show()

pre_dlab_ods_streak_steadio_outreach.printSchema()

+--------+
|rep_name|
+--------+
|    sean|
|    june|
|    cole|
| jaewook|
+--------+

+-----------+
|category_id|
+-----------+
|       9001|
|       null|
|       9007|
|       9005|
|       9006|
|       9008|
|       9004|
|       9003|
|       9023|
|       9002|
|       9021|
|       9022|
+-----------+

+-------------+
|stage_type_id|
+-------------+
|         5009|
|         5005|
|         5010|
|         5001|
|         5003|
|         5004|
|         5007|
+-------------+

+-----------+
|is_tumblbug|
+-----------+
|       null|
|       true|
|      false|
+-----------+

+-------------+
|leadsource_id|
+-------------+
|         9001|
|         null|
|         9002|
+-------------+

root
 |-- rep_name: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- stage_type_id: string (nullable = true)
 |-- is_tumblbug: boolean (nullable = true)
 |-- leadsource_id: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)


In [100]:
pre_dlab_ods_streak_steadio_outreach = DynamicFrame.fromDF(pre_dlab_ods_streak_steadio_outreach
                                                           , glue_context
                                                           , "pre_dlab_ods_streak_steadio_outreach");

In [103]:
pre_action = "TRUNCATE TABLE dlab.temp_streak_job"
pre_dlab_temp_streak_job = glue_context.write_dynamic_frame.from_jdbc_conf(
    frame=pre_dlab_ods_streak_steadio_outreach,
    catalog_connection="steadio-segment-con",
    connection_options={
        "database": "dev",
        "dbtable": "dlab.temp_streak_job",
        "preactions": pre_action
    },
    redshift_tmp_dir="s3://steadio-glue-info/temporary/"
)